# Preprocess Giant Data

#### Load data From csv files

In [1]:
import os
import pandas as pd
import numpy as np

directory = './Result/'
files = os.listdir(directory)

df = pd.concat([pd.read_csv(directory + f, 
                            usecols=["Gender", "BirthDate", "AdmissionDate", "DischargeDate", "Postcode", "Drg", "Cpt", "ServiceDate", "Charges"],
                            dtype={'Postcode': 'str', 'Drg': 'str', 'Cpt': 'str', 'BirthDate': 'str'},
                           ) for f in files], ignore_index = True)

#### Create postcode dictionary

In [2]:
file = './data/free-zipcode-database.csv'
df_postcode = pd.read_csv(file, usecols=["Zipcode", "State"])
pos_dict = df_postcode.to_dict(orient='split')
pos_dictionaray = {}
for pos_list in pos_dict['data']:
    pos_dictionaray[str(pos_list[0])] = pos_list[1]


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119042455 entries, 0 to 119042454
Data columns (total 9 columns):
Gender           object
BirthDate        object
AdmissionDate    object
DischargeDate    object
Postcode         object
Drg              object
Cpt              object
ServiceDate      object
Charges          object
dtypes: object(9)
memory usage: 8.0+ GB


#### Filter and Parse columns

In [4]:
def parseZip(zipcode):
    first_five = zipcode[:5]
    try:
        #   add leading zeroe
        code = '{:05}'.format(int(first_five))
        result = pos_dictionaray[code]
        if result:
            return result
        return 'null'
    except:
        return 'null'

In [5]:
def parseYear(date):
    try:
        year = date.split('/')[2]
    except:
        return date
    return year

In [6]:
def parseCharges(charge):
    new_charge = [x for x in charge if x.isdigit() or x == '.']
    return float("".join(new_charge))
    

In [7]:
df['Postcode'] = df['Postcode'].apply(lambda row: parseZip(row))
df['ServiceDate'] = df['ServiceDate'].apply(lambda row: parseYear(row))
df['Charges'] = df['Charges'].apply(lambda row: parseCharges(row))

In [8]:
group_by_year = df.groupby(['ServiceDate', 'Gender']).size()
group_by_postcode = df.groupby(['ServiceDate','Postcode']).size()
group_by_charges = df.groupby(['ServiceDate','Postcode']).sum()

#### Convert pandas sequence to python dictionary

In [9]:
gender_mapping = group_by_year.to_dict()

In [10]:
state_mapping = group_by_postcode.to_dict()

In [11]:
charges_mapping = group_by_charges.to_dict()
charges_mapping = charges_mapping['Charges']

##### Process dictionaries into one big dictionary

In [12]:
import operator

giant_data = {}
# do heat map first
HEAT_MAP = 'heatmap'
for key_tuple in state_mapping:
    year = key_tuple[0].strip()
    state = ("US-"+key_tuple[1].strip()).upper()
    if year not in giant_data:
        giant_data[year] = dict()
    if HEAT_MAP not in giant_data[year]:
        giant_data[year][HEAT_MAP] = dict()
    if state not in giant_data[year][HEAT_MAP]:
        giant_data[year][HEAT_MAP][state] = 0
    giant_data[year][HEAT_MAP][state] += int(state_mapping[key_tuple])
    
#then do stats
STATS = 'stats'
TOTAL_CASES = 'totalCases'
for key_tuple in gender_mapping:
    year = key_tuple[0]
    gender = key_tuple[1]
    if year not in giant_data:
        giant_data[year] = dict()
    if STATS not in giant_data[year]:
        giant_data[year][STATS] = dict()
        giant_data[year][STATS][TOTAL_CASES] = 0
        giant_data[year][STATS]['M'] = 0
        giant_data[year][STATS]['F'] = 0
    giant_data[year][STATS][TOTAL_CASES] += int(gender_mapping[key_tuple])
    giant_data[year][STATS][gender] += int(gender_mapping[key_tuple])

#Add ACPC
HEAT_MAP_ACPC = 'heatmapAcpc'
for key_tuple in charges_mapping:
    year = key_tuple[0]
    state = ("US-"+key_tuple[1].strip()).upper()
    if year not in giant_data:
        giant_data[year] = dict()
    if HEAT_MAP_ACPC not in giant_data[year]:
        giant_data[year][HEAT_MAP_ACPC] = dict()
    if state not in giant_data[year][HEAT_MAP_ACPC]:
        giant_data[year][HEAT_MAP_ACPC][state] = 0
    giant_data[year][HEAT_MAP_ACPC][state] += float(charges_mapping[key_tuple]) / giant_data[year][HEAT_MAP][state]

    
    
#Get top 5
TOP_HIGH = 'topHigh'
TOP_LOW = 'topLow'
TOP_HIGH_ACPC = 'topHighAcpc'
TOP_LOW_ACPC = 'topLowAcpc'
DELETED_KEY = 'US-NULL'
for year in giant_data:
    states = dict(giant_data[year][HEAT_MAP])
    if DELETED_KEY in states:
        del states[DELETED_KEY]
        
    sorted_states = sorted(states.items(), key=operator.itemgetter(1))
    giant_data[year][STATS][TOP_LOW] = sorted_states[:5]
    giant_data[year][STATS][TOP_HIGH] = sorted_states[::-1][:5]
    
    acpc = dict(giant_data[year][HEAT_MAP_ACPC])
    if DELETED_KEY in states:
        del states[DELETED_KEY]
    
    sorted_acpc = sorted(acpc.items(), key=operator.itemgetter(1))
    giant_data[year][STATS][TOP_LOW_ACPC] = sorted_acpc[:5]
    giant_data[year][STATS][TOP_HIGH_ACPC] = sorted_acpc[::-1][:5]



#### Export Dictionary to json file

In [13]:
import json
with open ('giant_data_file.json','w+') as f:
    json.dump(giant_data,f)